In [1]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

file1_text = extract_text_from_pdf('docs/9241562579.pdf')
file2_text = extract_text_from_pdf('docs/wcms_108221.pdf')
file3_text = extract_text_from_pdf('docs/youth-mental-health.pdf')
print("done")

done


In [5]:
import re

def preprocess_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

file1_text = preprocess_text(file1_text)
file2_text = preprocess_text(file2_text)
file3_text = preprocess_text(file3_text)

print("done")

done


In [6]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

# Combine text from the three documents
corpus = file1_text + file2_text + file3_text

# Tokenize the corpus
tokenized_corpus = tokenizer.encode(corpus, return_tensors="pt")


C:\Users\tehre\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tehre\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Token indices sequence length is longer than the specified maximum sequence length for this model (64533 > 1024). Running this sequence through the model will result in indexing errors


ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.

In [ ]:
def get_response(query):
    response = chatbot.get_response(query)
    return response

# Example interaction
user_input = "What are some mental health tips for the workplace?"
response = get_response(user_input)
print(response)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch

# Load pre-trained model and tokenizer
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    overwrite_output_dir=True,
)

# Prepare dataset (replace this with your own dataset)
train_dataset = [
    {"input_ids": tokenizer.encode("How are you?", return_tensors="pt"), "labels": tokenizer.encode("I'm fine, thank you.", return_tensors="pt")},
    {"input_ids": tokenizer.encode("What's your name?", return_tensors="pt"), "labels": tokenizer.encode("My name is Chatbot.", return_tensors="pt")},
    # Add more examples...
]

# Define data collator
def collate_fn(examples):
    return tokenizer.pad(examples, return_tensors='pt')

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-model")
